##이름이 있는 텐서

우리가 다루는 텐서는 차원이나 축이 있으며, 각 차원은 픽셀 위치나 컬러 채널에 해당한다. 때문에 텐서를 접근하려면 차원의 순서를 기억해서 인덱싱해야 한다. 데이터가 여러 텐서 형태를 거치며 다양하게 변환되면, 어느 차원에 어떤 데이터가 들어있는지 헷갈려 실수하기 쉽다.

In [1]:
#이미지 데이터를 흑백으로 변환해야 한다고 가정해보자. 여러 색상별 가중치를 보고 하나의 밝기 값을 뽑아내는 과정이 된다.

import torch

img_t = torch.randn(3, 5, 5) #채널 크기, 행 크기, 열 크기기
weights = torch.tensor([0.2126, 0.7152, 0.0722])

#일반화.
 높이와 너비를 가진 2차원 텐서로 이뤄진 흑백 이미지로부터 RGB값을 담을 세 번째 채널 차원을 더하는 코드로 만드든 것. 아니면 개별 이미지 여러 개를 묶어 배치로 만드는 코드

In [2]:
#배치 크기 2로 가정
batch_t = torch.randn(2,3,5,5)

In [3]:
#RGB 채널은 -3번 차원에 있는 것으로 일반화할 수 있다.

#따라서 다음 코드로 배치 차원 유무에 상관없이 평균을 구할 수 있다.
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [10]:
batch_gray_naive

tensor([[[-0.2293,  0.0916,  0.0190, -0.4202,  0.0801],
         [ 0.3998,  0.2165,  0.4545, -0.3524, -1.1498],
         [-0.7529,  0.3832, -0.0463, -0.1687, -0.9181],
         [-0.1749, -0.1172, -0.1508, -0.1260,  0.0680],
         [-0.4486,  1.1771,  0.5483,  0.5374, -0.4810]],

        [[-0.9432,  0.1063, -0.3839,  0.4028, -0.1157],
         [-1.5479,  0.8114,  0.7470,  0.4229, -0.3948],
         [-0.7439,  0.3172,  0.0022,  0.3902, -0.4185],
         [-1.1391, -0.2691, -0.1470,  0.4822, -1.3304],
         [-0.4860, -0.3038,  0.1404, -0.0097, -0.0693]]])

지금은 가중치가 있다. 파이토치는 동일한 차원 정보의 텐서끼리 연산할 수 도 있고, 각 차원의 길이가 1인 텐서도 가능하다 
혹은 길이가 1인 차원을 알아서 늘려주기도 하는데, 이런 방식을 broadcasting이라 한다.
즉, (2,3,5,5,) 차원을 가진 batch_t를 (3,1,1) 차원의 unsqueezed_weights로 곱하면 (2,3,5,5) 차원이 되고, 채널 정보를 가진, 뒤에서 세 번째 차원에 있는 값에 대한 합을 구할 수 있다.

In [11]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)

img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)

img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)

batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

#unsqueeze함수는 squeeze함수의 반대로 1인 차원을 생성하는 함수이다. 그래서 어느 차원에 1인 차원을 생성할 지 꼭 지정해주어야한다.

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [12]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

#einsum함수

torch.Size([2, 5, 5])

In [14]:
#각 차원에 이름 부여하기
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [15]:
#refine 함수
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


텐서끼리의 연산은 먼저 각 차원의 크기가 같은지 혹은 한쪽이 1이고 다른 쪽으로 브로드캐스팅될 수 있는지도 확인해야 한다. 이때 이름이 지정되어 있다면 파이토치가 우리를 대신해 알아서 체크해줄 것이다.

파이토치가 차원을 자동으로 정렬해주지는 않기에 우리는 명시적으로 이런 작업을 수행할 필요가 있다. align_as 함수는 빠진 차원을 채우고, 존재하는 차원을 올바를 순서로 바꿔준다.

In [16]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [17]:
#sum처럼 차원 인수를 허용하는 함수들은 이름이 붙은 차원도 받아들인다.
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [18]:
#이름이 다른 차원을 결합하려면 오류가 발생한다.
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [19]:
#이름 있는 텐서를 사용하는 연산을 함수 밖에서도 사용하려면, 다음처럼 차원 이름에 None을 넣어 이름 없는 텐서를 만든다.
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))